In [ ]:
import pandas as pd
import numpy as np
from src.util import load_sentences, preprocess_sentences

import logging
import pandas as pd
import sklearn
import wandb

### Load Datasets (Train, Val, Test)

In [ ]:
# Dataset
base_path = ''
train_samples = load_sentences(base_path + 'test.txt')
val_samples = load_sentences(base_path + 'valid.txt')
test_samples = load_sentences(base_path + 'test.txt')

train_samples = np.random.choice(train_samples, 100, False)
val_samples = np.random.choice(val_samples, 10, False)
test_samples = np.random.choice(test_samples, 100, False)

df_train = preprocess_sentences(train_samples)
df_val = preprocess_sentences(val_samples)
df_test = preprocess_sentences(test_samples)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarr

### Sweep Config

In [ ]:

sweep_config = {
    "method": "random",  # grid, random
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"values": [1,2,3]},
        "learning_rate": {"min": 5e-5, "max": 4e-4},
        "train_batch_size": {"values": [16, 32]},
    }
}

sweep_id = wandb.sweep(sweep_config, project="bert_ner6")
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: avenl9ux
Sweep URL: https://wandb.ai/vinicezarml/bert_ner6/sweeps/avenl9ux


### Train

In [ ]:

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel,NERArgs

args = NERArgs()
args.overwrite_output_dir =True
args.evaluate_during_training_steps = 10
args.save_model_every_epoch = False


def train():

    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    # Choose a pre-trained model: https://huggingface.co/models?sort=downloads
    labels_list = df_train["labels"].unique().tolist()
    model = NERModel('bert', 'dslim/bert-base-NER', labels=labels_list, args=args, use_cuda=False)

    # Train the model
    model.train_model(df_train, eval_df=df_val, show_running_loss=True)

    # Evaluate the model
    result, model_outputs, preds_list = model.eval_model(df_test)

    # Log results
    wandb.log({"eval_loss": result["eval_loss"]
               , "precision": result["precision"]
               , "recall": result["recall"]
               , "f1_score": result["f1_score"]})

    # Sync wandb
    wandb.join()

wandb.agent(sweep_id, train, count=20)


wandb: Agent Starting Run: xcducr82 with config:
wandb: 	learning_rate: 0.0001920559302207583
wandb: 	num_train_epochs: 1
wandb: 	train_batch_size: 16


Downloading:   0%|          | 0.00/829 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

INFO:simpletransformers.ner.ner_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.ner.ner_model: Converting to features started.


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

INFO:simpletransformers.ner.ner_model:{'eval_loss': 0.7452040658547328, 'precision': 0.2028301886792453, 'recall': 0.25903614457831325, 'f1_score': 0.22751322751322753}


eval_loss,▁
f1_score,▁
precision,▁
recall,▁
eval_loss,0.7452
f1_score,0.22751
precision,0.20283
recall,0.25904


### Dashboard with the training logs
Here: https://wandb.ai/vinicezarml/bert_ner6/reports/NER-Experiment-Results--VmlldzoxMzE0ODUw

In [1]:
import IPython
iframe = '<iframe src="https://wandb.ai/vinicezarml/bert_ner6/reports/NER-Experiment-Results--VmlldzoxMzE0ODUw" style="border:none;height:1024px;width:100%">'
IPython.display.HTML(iframe)
